In [35]:
import astropy
from astropy.table import Table, Column
from astroquery.xmatch import XMatch

In [57]:
def xMatch(cat1, cat2, 
           colNameX1='RA', colNameX2='RA', colNameY1='DEC', colNameY2='DEC', 
           cartesian=False,
           minimum_value=99e99
          ):
    # oh god, I have to learn astropy.table methods, don't I? :(
    newTable = Table(Column(name="xMatchR", unit=None, description="xMatch distance"), 
                     cat1.columns, cat2.columns,
                     masked=None)
    for row in cat1:
        Rmin = minimum_value
        best_match = None

        for crow in cat2:
            if cartesian is True:
                Rtest = np.sqrt(np.square(row[colNameX1] - crow[colNameX2]) +
                                np.square(row[colNameY1] - crow[colNameY2]))
            else:
                # Convert from degrees.
                f1, l1 = (row[colNameX1] * np.pi / 180.0, row[colNameY1] * np.pi / 180.0)
                f2, l2 = (crow[colNameX2] * np.pi / 180.0, crow[colNameY2] * np.pi / 180.0)
                num = np.sqrt(np.square(np.cos(f2) * np.sin(l2 - l1)) +
                              np.square(np.cos(f1) * np.sin(f2) - np.sin(f1) * np.cos(f2) * np.cos(l2 - l1)))
                den = np.sin(f1) * np.sin(f2) + np.cos(f1) * np.cos(f2) * np.cos(l2 - l1)
                Rtest = np.atan2(num, den) * 180.0 / np.pi * 3600.0

            if Rtest < Rmin:
                Rmin = Rtest
                best_match = crow
        if best_match is None:
            newTable.add_row(None, row, None * len(cat2.columns))
        else:
            newTable.add_row(Rmin, row, best_match)
    return newTable

In [54]:
# https://heasarc.gsfc.nasa.gov/W3Browse/all/mwsc.html
mwsc = Table.read("./data/mwsc_catalog.fits", hdu='MWSC')

In [55]:
mwgc = Table.read("./data/mwgc_catalog.fits", hdu='GLOBCLUST')

In [56]:
joint = xMatch(mwsc, mwgc, minimum_value=10.0)

TypeError: __init__() got multiple values for argument 'masked'

In [ ]:
print(joint)

In [32]:
help(XMatch)

Help on XMatchClass in module astroquery.xmatch.core object:

class XMatchClass(astroquery.query.BaseQuery)
 |  This is the base class for all the query classes in astroquery. It
 |  is implemented as an abstract class and must not be directly instantiated.
 |  
 |  Method resolution order:
 |      XMatchClass
 |      astroquery.query.BaseQuery
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  get_available_tables(self, cache=True)
 |      Get the list of the VizieR tables which are available in the
 |      xMatch service and return them as a list of strings.
 |  
 |  is_table_available(self, table_id)
 |      Return True if the passed CDS table identifier is one of the
 |      available VizieR tables, otherwise False.
 |  
 |  query(self, cat1, cat2, max_distance, colRA1=None, colDec1=None, colRA2=None, colDec2=None, cache=True, get_query_payload=False)
 |      Query the `CDS cross-match service
 |      <http://cdsxmatch.u-strasbg.fr/xmatch>`_ by finding matches between
